In [4]:
import torch
import torchvision
from torchvision import datasets,transforms
import matplotlib.pyplot as plt
import numpy as np
import cv2
 
import torch.nn as nn
import torch.nn.functional as F

import tensorboard

In [5]:
tb_logger

NameError: name 'tb_logger' is not defined

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
 
transform = transforms.Compose([
        transforms.ToTensor(), # 转为Tensor
        transforms.Normalize((0.5,), (0.5,)), # 归一化
                             ])
 
train_dataset = torchvision.datasets.MNIST(root='./mnist', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist', train=False, transform=transform, download=True)
 
batch_size = 4
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5)  
        self.fc1   = nn.Linear(16*5*5, 120)  
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)
 
    def forward(self, x): 
        # print(x.size())    # 4-1-28-28
        
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2)) 
        # print(x.size())    # # 4-6-14-14
        
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        # print("x.size():", x.size())   # 4-16-5-5
        
        x = x.view(x.size()[0], -1)   #展开成一维的
        # print("x.size():", x.size())   # 4-400
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)        
        return x

In [9]:
net = Net()
print(net)
 
from torch import optim
criterion = nn.CrossEntropyLoss() # 交叉熵损失函数
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # 优化器
 
import time
 
start_time = time.time()

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [10]:
for epoch in range(10):
    running_loss = 0.0 #初始化loss
    for i, (inputs, labels) in enumerate(trainloader, 0):
 
        # 输入数据
        inputs =  inputs.to(device)
        labels =  labels.to(device)
 
        # 梯度清零
        optimizer.zero_grad()
 
        # forward + backward 
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()   
 
        # 更新参数 
        optimizer.step()
 
        # 打印log信息
        # loss 是一个scalar,需要使用loss.item()来获取数值，不能使用loss[0]
        running_loss += loss.item()
        if i % 2000 == 1999: # 每2000个batch打印一下训练状态
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss / 2000))
            running_loss = 0.0

stop_time = time.time()
print('Finished Training 耗时： ', (stop_time - start_time), '秒')
torch.save(net.state_dict(), "Lenet5_1000.pth")

[1,  2000] loss: 1.006
[1,  4000] loss: 0.221
[1,  6000] loss: 0.135
[1,  8000] loss: 0.117
[1, 10000] loss: 0.099
[1, 12000] loss: 0.093
[1, 14000] loss: 0.084
[2,  2000] loss: 0.073
[2,  4000] loss: 0.071
[2,  6000] loss: 0.058
[2,  8000] loss: 0.060
[2, 10000] loss: 0.059
[2, 12000] loss: 0.055
[2, 14000] loss: 0.053
[3,  2000] loss: 0.047
[3,  4000] loss: 0.045
[3,  6000] loss: 0.039
[3,  8000] loss: 0.040
[3, 10000] loss: 0.042
[3, 12000] loss: 0.038
[3, 14000] loss: 0.037
[4,  2000] loss: 0.036
[4,  4000] loss: 0.035
[4,  6000] loss: 0.027
[4,  8000] loss: 0.030
[4, 10000] loss: 0.034
[4, 12000] loss: 0.029
[4, 14000] loss: 0.030
[5,  2000] loss: 0.025
[5,  4000] loss: 0.025
[5,  6000] loss: 0.023
[5,  8000] loss: 0.025
[5, 10000] loss: 0.029
[5, 12000] loss: 0.023
[5, 14000] loss: 0.024
[6,  2000] loss: 0.022
[6,  4000] loss: 0.023
[6,  6000] loss: 0.017
[6,  8000] loss: 0.020
[6, 10000] loss: 0.026
[6, 12000] loss: 0.020
[6, 14000] loss: 0.022
[7,  2000] loss: 0.017
[7,  4000] 

In [53]:
#整个测试集上预测
correct = 0
total = 0
 
with torch.no_grad():
    for (images,labels) in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs,1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum()

        print('10000张测试集合中的准确率为：', (correct.cpu().numpy()/total * 100))
print(correct)

SyntaxError: invalid character in identifier (1337901149.py, line 17)

In [47]:

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')
 
#**测试图像的实际labels**
dataiter = iter(testloader) #把测试数据放在迭代器iter
images, labels = dataiter.next() # 一个batch返回4张图片，依次获取下一个数据
images = images.to(device)
labels = labels.to(device)
print('实际的label: ', ' '.join( '%08s'%classes[labels[j]] for j in range(4)))
 
print(images/2+0.5)
img = np.empty((28,28*4), dtype=np.float32)
img[:,0:28] = images[0].numpy()
img[:,28:56] = images[1].numpy()
img[:,56:84] = images[2].numpy()
img[:,84:112] = images[3].numpy()
 
img2 = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
print(img2.shape)
plt.imshow(img2)
plt.show()
————————————————
版权声明：本文为CSDN博主「不解不惑」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/qq_24815615/article/details/105207630

SyntaxError: invalid character in identifier (2081364254.py, line 21)

In [ ]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
    def forward(self, x):
        x.view(x.size()[0], -1)


class Net_LeNet(nn.Module):
    def __init__(self):
        super(Net_LeNet, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1, 6, 5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),
            
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),
        )
        
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.layers(x)
        # print(x.size())   # 4-16-5-5
        x = x.view(x.size()[0], -1)
        # 4-400
        x = F.relu(self.fc1(x))
        # print(x.size())   # 4-120
        x = F.relu(self.fc2(x))
        # print(x.size())    # 4-84
        x = self.fc3(x)
        
        return x